In [2]:
from scipy.ndimage import zoom
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

In [3]:
def calculate_frequencies(indices, dt, signal_length):
    """Calculate and interpolate frequencies"""
    time_periods = np.diff(indices) * dt
    frequencies = 1 / time_periods
    midpoints = indices[:-1] + np.diff(indices) / 2
    f = interp1d(midpoints, frequencies, kind='linear', fill_value='extrapolate', bounds_error=False)
    return f(np.arange(signal_length))

def infer_frequencies(x, dt=1/N):
    """
    Assign current frequency at each timestep.
    
    Parameters:
       x(numpy.ndarray): input, continuous signal
       dt(float): time difference between each subsequent sample (1/Fs)
       
    Returns:
       numpy.ndarray: current frequency at each timestep
    """
    # Detect peaks and troughs
    peak_indices = find_peaks(x)[0]
    trough_indices = find_peaks(-x)[0]

    # Calculate frequencies using peaks and troughs independently
    peak_freqs = calculate_frequencies(peak_indices, dt, len(x))
    trough_freqs = calculate_frequencies(trough_indices, dt, len(x))

    # Combine the frequencies calculated from peaks and troughs
    combined_freqs = (peak_freqs + trough_freqs) / 2

    return combined_freqs

In [4]:
def get_angle(f):
    return np.angle(np.fft.fft(f)[1])

def get_angles(F):
    return np.angle(np.fft.fft(F, axis=1)[:, 1])

def get_amps(F):
    return np.abs(np.fft.fft(F, axis=1)[:, 1])

In [5]:
def to_theta(x):
    return ((x + np.pi) % (2*np.pi)) - np.pi

In [6]:
def mean_angle(amps, angs):
    s = sum(amps)
    return np.angle(np.sum([
        amp * np.exp(1j*ang)
        for amp, ang in zip(amps, angs)
    ], axis=0))